# Inferring Photometric Redshifts from Multichannel Images
## Balázs Menkó (O67UT7)
### Supervisor: Pál, Balázs 

Spectroscopic observations of distant astronomical targets are increasingly difficult and expensive to obtain. Therefore, it is crucial to develop methods for inferring the physical parameters of objects from photometric data, which is the only type of observation available at high redshifts. The Sloan Digital Sky Survey (SDSS) data set is a valuable resource for this task, as it contains both photometric and spectroscopic data for a very large number of galaxies.

Analyze the photometric and spectroscopic data of galaxies together! Download coordinates, redshifts and appropriate object identifiers from the SDSS SkyServer! Create square-shaped images from the SDSS sky survey observations that contain single galaxies in the center! Build a simple convolutional neural network that is able to infer redshifts from the cut-out images of galaxies!

#### How accurate is your model?

---

Change **Kooplex Environment** image to `image-registry.vo.elte.hu/jupyter-tensorflow-v6` and set `CPU [#]` to the maximum.

Install `psycopg2`:

```python
%%bash
pip install psycopg2
```

In [ ]:
### PACKAGES
from utils import *

import tensorflow as tf
import tensorflow.keras.layers as tf_kl
import tensorflow.keras.regularizers as tf_kr
tf.config.threading.set_intra_op_parallelism_threads(8)

print("Num GPUs Available: ", len(tf.config.experimental.list_physical_devices('GPU')))

In [ ]:
number_of_pictures = 8000
epochs = 50
batch_size = 128

In [ ]:
np.random.seed(137)
indices = np.arange(0, number_of_pictures)
np.random.shuffle(indices)
train_set_indices = indices[:int(len(indices)*0.5)]
test_set_indices = indices[int(len(indices)*0.5):int(len(indices)*0.8)]
#validation_set_indices = indices[int(len(indices)*0.8):]

train_set_indices = '), ('.join(map(str, list(train_set_indices)))
test_set_indices = '), ('.join(map(str, list(test_set_indices)))
#validation_set_indices = '), ('.join(map(str, list(validation_set_indices)))

In [ ]:
train_set = query_a_needed_set(train_set_indices)
test_set = query_a_needed_set(test_set_indices)
#validation_set = query_a_needed_set(validation_set_indices)

In [ ]:
column='picture'
train_set_pictures = get_data(train_set, column)
test_set_pictures = get_data(test_set, column)
#validation_set_pictures = get_data(validation_set, column)

In [ ]:
column='z'
train_set_features = get_data(train_set, column)
test_set_features = get_data(test_set, column)
#validation_set_features = get_data(validation_set, column)

---
# Create and train model

In [ ]:
filters=32
kernel_size=(3,3)
padding='same'
activation='relu'
regularization=5e-5


model_sdss = tf.keras.models.Sequential([ 
    # Input
    tf_kl.Input(shape=(train_set_pictures.shape[1:]), name='Input_layer'),
    
    # Convolutional block 1 --- 3x3CONVxfltr -> ReLU -> 3x3CONVxfltr -> ReLU -> MAXPOOL2x2
        tf_kl.Conv2D(filters=filters, kernel_size=kernel_size, padding=padding, 
                     kernel_regularizer=tf_kr.l2(regularization), name='Conv2D_layer-1'),
        tf_kl.BatchNormalization(name='BatchNorm_layer-2'),
        tf_kl.Activation(activation, name='Relu_layer-3'),
        tf_kl.Conv2D(filters=filters, kernel_size=kernel_size, padding=padding, 
                     kernel_regularizer=tf_kr.l2(regularization), name='Conv2D_layer-4'),
        tf_kl.BatchNormalization(name='BatchNorm_layer-5'),
        tf_kl.Activation(activation, name='Relu_layer-6'),
        tf_kl.MaxPool2D(strides=(2,2), name='MaxPool2D_layer-7'),
    
    # Convolutional block 2 --- 3x3CONVx2*fltr -> ReLU -> 3x3CONVx2*fltr -> ReLU -> MAXPOOL2x2
        tf_kl.Conv2D(filters=2*filters, kernel_size=kernel_size, padding=padding, 
                     kernel_regularizer=tf_kr.l2(regularization), name='Conv2D_layer-8'),
        tf_kl.BatchNormalization(name='BatchNorm_layer-9'),
        tf_kl.Activation(activation, name='Relu_layer-10'),
        tf_kl.Conv2D(filters=2*filters, kernel_size=kernel_size, padding=padding, 
                     kernel_regularizer=tf_kr.l2(regularization), name='Conv2D_layer-11'),
        tf_kl.BatchNormalization(name='BatchNorm_layer-12'),
        tf_kl.Activation(activation, name='Relu_layer-13'),
        tf_kl.MaxPool2D(strides=(2,2), name='MaxPool2D_layer-14'),
    
    # Convolutional block 3 --- 3x3CONVx4*fltr -> ReLU -> 1x1CONVx2*fltr -> ReLU -> 3x3CONVx4*fltr -> ReLU -> MAXPOOL2x2
        tf_kl.Conv2D(filters=4*filters, kernel_size=kernel_size, padding=padding, 
                     kernel_regularizer=tf_kr.l2(regularization), name='Conv2D_layer-15'),
        tf_kl.BatchNormalization(name='BatchNorm_layer-16'),
        tf_kl.Activation(activation, name='Relu_layer-17'),
        tf_kl.Conv2D(filters=2*filters, kernel_size=(1,1), padding=padding, 
                     kernel_regularizer=tf_kr.l2(regularization), name='Conv2D_layer-18'),
        tf_kl.BatchNormalization(name='BatchNorm_layer-19'),
        tf_kl.Activation(activation, name='Relu_layer-20'),
        tf_kl.Conv2D(filters=4*filters, kernel_size=kernel_size, padding=padding, 
                     kernel_regularizer=tf_kr.l2(regularization), name='Conv2D_layer-21'),
        tf_kl.BatchNormalization(name='BatchNorm_layer-22'),
        tf_kl.Activation(activation, name='Relu_layer-23'),
        tf_kl.MaxPool2D(strides=(2,2), name='MaxPool2D_layer-24'),
    
    # Convolutional block 4 --- 3x3CONVx8*fltr -> ReLU -> 1x1CONVx4*fltr -> ReLU -> 3x3CONVx8*fltr -> ReLU -> MAXPOOL2x2
        tf_kl.Conv2D(filters=8*filters, kernel_size=kernel_size, padding=padding, 
                     kernel_regularizer=tf_kr.l2(regularization), name='Conv2D_layer-25'),
        tf_kl.BatchNormalization(name='BatchNorm_layer-26'),
        tf_kl.Activation(activation, name='Relu_layer-27'),
        tf_kl.Conv2D(filters=4*filters, kernel_size=(1,1), padding=padding, 
                     kernel_regularizer=tf_kr.l2(regularization), name='Conv2D_layer-28'),
        tf_kl.BatchNormalization(name='BatchNorm_layer-29'),
        tf_kl.Activation(activation, name='Relu_layer-30'),
        tf_kl.Conv2D(filters=8*filters, kernel_size=kernel_size, padding=padding, 
                     kernel_regularizer=tf_kr.l2(regularization), name='Conv2D_layer-31'),
        tf_kl.BatchNormalization(name='BatchNorm_layer-32'),
        tf_kl.Activation(activation, name='Relu_layer-33'),
        tf_kl.MaxPool2D(strides=(2,2), name='MaxPool2D_layer-34'),
    
    # Convolutional block 5. --- 3x3CONVx16*fltr -> ReLU -> 1x1CONVx8*fltr -> ReLU -> 3x3CONVx16*fltr -> ReLU -> AVGPOOL
        tf_kl.Conv2D(filters=16*filters, kernel_size=kernel_size, padding=padding, 
                     kernel_regularizer=tf_kr.l2(regularization), name='Conv2D_layer-35'),
        tf_kl.BatchNormalization(name='BatchNorm_layer-36'),
        tf_kl.Activation(activation, name='Relu_layer-37'),
        tf_kl.Conv2D(filters=8*filters, kernel_size=(1,1), padding=padding, 
                     kernel_regularizer=tf_kr.l2(regularization), name='Conv2D_layer-38'),
        tf_kl.BatchNormalization(name='BatchNorm_layer-39'),
        tf_kl.Activation(activation, name='Relu_layer-40'),
        tf_kl.Conv2D(filters=16*filters, kernel_size=kernel_size, padding=padding, 
                     kernel_regularizer=tf_kr.l2(regularization), name='Conv2D_layer-41'),
        tf_kl.BatchNormalization(name='BatchNorm_layer-42'),
        tf_kl.Activation(activation, name='Relu_layer-43'),
        tf_kl.GlobalAveragePooling2D(name='GlobAvgPool_layer-44'),  
    
    # End of convolution
        tf_kl.Dense(1, name='Output_layer')],
    
    name="Model-SDSS")

model_sdss.compile(loss='mean_squared_error',
                   metrics={'Output_layer': ['mean_absolute_error']},
                   optimizer=tf.keras.optimizers.Adam(learning_rate=5e-5))
model_sdss.summary()

In [ ]:
%%time
history_sdss = model_sdss.fit(x = train_set_pictures,
                              y = train_set_features, 
                              validation_data = (test_set_pictures, test_set_features),
                              batch_size = batch_size, 
                              epochs = epochs
                             )

In [ ]:
now = datetime.now()

np.save(f"models/history-sdss_time={now.month}-{now.day}-{now.hour}-{now.minute}"+\
        f"_n-pic={number_of_pictures}_epoch={epochs}_batch-size={batch_size}.npy", 
        history_sdss.history)
tf.keras.models.save_model(
    model_sdss,  
    f"models/model-sdss_time={now.month}-{now.day}-{now.hour}-{now.minute}"+\
    f"_n-pic={number_of_pictures}_epoch={epochs}_batch-size={batch_size}.keras",
    overwrite=False,
)

---
# Continue training

### Trained models:
First run --- redshift --- $ N_{pic}=8000$, epochs$=50$, batch size$=128$, time=11-19-0-30

Second run --- redshift --- $ N_{pic}=8000$, epochs$=80$, batch size$=128$, time=11-20-19-26

In [ ]:
load_nth_model = 1  

if load_nth_model == 1:
    number_of_pictures = 8000
    str_epochs = 50
    str_batch_size = 128
    str_time='11-19-0-30'
elif load_nth_model == 2:
    number_of_pictures = 8000
    str_epochs = 80
    str_batch_size = 128
    str_time='11-20-19-26'   
else:
    print(f"There is no model with parameter 'load_nth_model={load_nth_model}.'")

In [ ]:
if False:

    model_sdss = tf.keras.models.load_model(
        f"models/model-sdss_time={str_time}"+\
        f"_n-pic={str_number_of_pictures}_epoch={str_epochs}_batch-size={str_batch_size}.keras",
    )

    if str_number_of_pictures == number_of_pictures:
        continue_tr_epochs=30
        history_sdss = model_sdss.fit(x = train_set_pictures,
                                      y = train_set_features, 
                                      validation_data = (test_set_pictures, test_set_features),
                                      batch_size = str_batch_size, 
                                      epochs = continue_tr_epochs
                                     )

    now = datetime.now()
    if str_number_of_pictures == number_of_pictures:
        np.save(f"models/history-sdss_time={now.month}-{now.day}-{now.hour}-{now.minute}"+\
                f"_n-pic={str_number_of_pictures}_epoch={str_epochs}+{continue_tr_epochs}_batch-size={batch_size}.npy", 
                history_sdss.history)
        tf.keras.models.save_model(
            model_sdss,  
            f"models/model-sdss_time={now.month}-{now.day}-{now.hour}-{now.minute}"+\
            f"_n-pic={str_number_of_pictures}_epoch={str_epochs}+{epochs}_batch-size={batch_size}.keras",
            overwrite=False,
        )
    

---
### Run times:
First run --- redshift --- $ N_{pic}=8000$, epochs$=50$, batch size$=128$, time=11-19-0-30
```python
# CPU times: user 17h 19min 31s, sys: 3h 5min 10s, total: 20h 24min 41s
# Wall time: 4h 10min 42s
```

Second run --- redshift --- $ N_{pic}=8000$, epochs$=80$, batch size$=128$, time=11-20-19-26
```python
# CPU times: user 21h 57min 43s, sys: 3h 31min 45s, total: 1d 1h 29min 29s
# Wall time: 5h 10min 41s
```